In [1]:
import random
import diff_classifier.aws as aws
import boto3
import skimage.io as sio
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn import svm

In [2]:
folder = '01_18_Experiment'

missing = []
for i in range(10, 15):
    missing.append("P1_S2_R_00{}".format(i))

for i in range(10, 15):
    missing.append("P2_S3_L_00{}".format(i))
    
for i in range(0, 15):
    missing.append("P3_S3_L_{}".format("%04d" % i))

pups = ["P3", "P2", "P1"]
slices = ["S1", "S2", "S3"]
folder = '01_18_Experiment'

hemis = ["R", "L"]
vids = 15

tnum=20 #number of training datasets

In [3]:
to_track = []

for pup in pups:
    for hemi in hemis:
            for slic in slices:
                for vid in range(0, vids):
                            prefix = "{}_{}_{}_{}".format(pup, slic, hemi, "%04d" % vid)
                            if not prefix in missing:
                                for row in range(0, 4):
                                    for col in range(0, 4):
                                        to_track.append("{}_{}_{}".format(prefix, row, col))

In [ ]:
len(to_track)

In [8]:
tprefix = []
for i in range(0, tnum):
    random.seed(i+1)
    tprefix.append(to_track[random.randint(0, len(to_track))])

In [ ]:
tprefix

In [4]:
y = np.array([1.5, 833.6, 9.24, 4.5, 3.3, 3.4, 2.85, 2.75, 3.7, 2.45,
              2.6, 3.1, 3.25, 2.85, 2.05, 2.9, 5.4, 5.0, 2.7, 530])

In [9]:
for name in tprefix:
    remote_folder = "01_18_Experiment/{}".format(name.split('_')[0])
    aws.download_s3(remote_folder+'/'+name+'.tif', name+'.tif')

In [ ]:
descriptors = np.zeros((tnum, 4))
counter = 0
for name in tprefix:
    local_im = name + '.tif'
    test_image = sio.imread(local_im)
    descriptors[counter, 0] = np.mean(test_image[0, :, :])
    descriptors[counter, 1] = np.std(test_image[0, :, :])
    descriptors[counter, 2] = np.percentile(test_image[0, :, :], 10)
    descriptors[counter, 3] = np.percentile(test_image[0:, :, :], 90)
    counter = counter + 1

In [ ]:
X = descriptors

classifiers = [
    svm.SVR(),
    linear_model.SGDRegressor(),
    linear_model.BayesianRidge(),
    linear_model.LassoLars(),
    linear_model.ARDRegression(),
    linear_model.PassiveAggressiveRegressor(),
    linear_model.TheilSenRegressor(),
    linear_model.LinearRegression()]

In [19]:
pto_track = to_track[1:5]
pX = np.zeros((len(pto_track), 4))
counter = 0
for name in pto_track:
    remote_folder = "01_18_Experiment/{}".format(name.split('_')[0])
    aws.download_s3(remote_folder+'/'+name+'.tif', name+'.tif')
    
    local_im = name + '.tif'
    test_image = sio.imread(local_im)
    pX[counter, 0] = np.mean(test_image[0, :, :])
    pX[counter, 1] = np.std(test_image[0, :, :])
    pX[counter, 2] = np.percentile(test_image[0, :, :], 10)
    pX[counter, 3] = np.percentile(test_image[0:, :, :], 90)
    counter = counter + 1

In [ ]:
for item in classifiers:
    print(item)
    clf = item
    clf.fit(X, y)
    print(clf.predict(pX), '\n')

In [20]:
pto_track[0]

'P3_S1_R_0000_0_1'

In [29]:
def regress_tracking_params(regress_object, to_track, regmethod='LinearRegression'):
    """
    Uses the regress object from regress_sys to predict tracking
    parameters for TrackMate.
    """
    
    local_im = to_track + '.tif'
    pX = np.zeros((1, 4))
    test_image = sio.imread(local_im)
    pX[0, 0] = np.mean(test_image[0, :, :])
    pX[0, 1] = np.std(test_image[0, :, :])
    pX[0, 2] = np.percentile(test_image[0, :, :], 10)
    pX[0, 3] = np.percentile(test_image[0:, :, :], 90)
    
    quality = []
    for item in regress_object:
        quality.append(item.predict(pX)[0])
    
    if regmethod == 'SVR':
        return quality[0]
    elif regmethod == 'SGDRegressor':
        return quality[1]
    elif regmethod == 'BayesianRidge':
        return quality[2]
    elif regmethod == 'LassoLars':
        return quality[3]
    elif regmethod == 'ARDRegression':
        return quality[4]
    elif regmethod == 'PassiveAggressiveRegressor':
        return quality[5]
    elif regmethod == 'TheilSenRegressor':
        return quality[6]
    elif regmethod == 'LinearRegression':
        return quality[7]

In [30]:
regress_tracking_params(regress, pto_track[0], regmethod='PassiveAggressiveRegressor')

1.7388826406802635

In [13]:
def regress_sys(all_videos, y, training_size, have_output=True):
    """
    Uses regression techniques to select the best tracking parameters.
    Regression again intensities of input images.
    """

    tprefix = []
    for i in range(0, tnum):
        random.seed(i+1)
        tprefix.append(all_videos[random.randint(0, len(all_videos))])
        if have_output is False:
            print("Get parameters for: {}".format(tprefix[i]))

    if have_output is True:
        # Define descriptors
        descriptors = np.zeros((tnum, 4))
        counter = 0
        for name in tprefix:
            local_im = name + '.tif'
            test_image = sio.imread(local_im)
            descriptors[counter, 0] = np.mean(test_image[0, :, :])
            descriptors[counter, 1] = np.std(test_image[0, :, :])
            descriptors[counter, 2] = np.percentile(test_image[0, :, :], 10)
            descriptors[counter, 3] = np.percentile(test_image[0:, :, :], 90)
            counter = counter + 1

        # Define regression techniques
        X = descriptors
        classifiers = [
            svm.SVR(),
            linear_model.SGDRegressor(),
            linear_model.BayesianRidge(),
            linear_model.LassoLars(),
            linear_model.ARDRegression(),
            linear_model.PassiveAggressiveRegressor(),
            linear_model.TheilSenRegressor(),
            linear_model.LinearRegression()]

        regress_object = []
        for item in classifiers:
            clf = item
            regress_object.append(clf.fit(X, y))
        
        return regress_object

In [14]:
regress = regress_tracking_params(to_track, y, tnum)

/home/ubuntu/miniconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/ubuntu/miniconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.passive_aggressive.PassiveAggressiveRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [12]:
local_im = tprefix[0]+'.tif'
test_image = sio.imread(local_im)

In [26]:
regress

[SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
   kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
 SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
        fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
        loss='squared_loss', max_iter=None, n_iter=None, penalty='l2',
        power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
        warm_start=False),
 BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
        fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
        normalize=False, tol=0.001, verbose=False),
 LassoLars(alpha=1.0, copy_X=True, eps=2.220446049250313e-16,
      fit_intercept=True, fit_path=True, max_iter=500, normalize=True,
      positive=False, precompute='auto', verbose=False),
 ARDRegression(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
        fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=